In [ ]:
#make the data augmentation
import numpy as np
import pandas as pd
import sklearn
from tqdm.notebook import tqdm

# from nlpaug.util.file.download import DownloadUtil
# DownloadUtil.download_fasttext(model_name='wiki-news-300d-1M', dest_dir='.') # Download fasttext model
#DownloadUtil.download_word2vec(dest_dir='.') # Download word2vec model

df = pd.read_csv('fixed.csv')

#transform the labels in numbers
int2label = {}

for i, disease in enumerate(df['label'].unique()):
    int2label[i] = disease

label2int = {v : k for k, v in int2label.items()}
num_classes = len(int2label)

#transform all the labels in integers
df['label'] = df['label'].map(lambda x : label2int[x])

#separate train and validation data for augmentation
from sklearn.model_selection import train_test_split

train,valid=train_test_split(df,test_size=0.15)
print('Shape of train before augmentation: ',train.shape)
print('Shape of Validation: ', valid.shape)

valid.to_csv("validation-data.csv", sep=',', encoding='utf-8', index=False)



import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from sklearn.utils import shuffle
from tqdm import tqdm
from nlpaug.util import Action

#this functions receive the dataset, the label and a number of samples (default is 300)
#make the data augmentation
import random
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from sklearn.utils import shuffle
from tqdm import tqdm
from nlpaug.util import Action

#this functions receive the dataset, the label and a number of samples (default is 300)
def augment_text(df,label,samples=300):

  #create the model for augmentation
  #here we use word2vec
  #the model is used for word substitution, selecting some random words and substiting to similar words
  aug = naw.RandomWordAug(action="swap")

  #array of texts and labels
  new_texts = []
  labels = []

  #separate the current label for augmentation
  df_n = df[df.label==label].reset_index(drop=True)
  print('Original shape of class',df_n.shape)

  #for each row in the original dataset, and for all samples given, make the augmentation
  for i in tqdm(np.random.randint(0,len(df_n),samples)):
    #locate the current row in the dataset, and separate label and symptom
    symptom = df_n.iloc[i]['text']
    labels.append(df.iloc[i]['label'])

    #make the augmentation and add to the array
    augmented_text_data = aug.augment(symptom)

    new_texts.append(augmented_text_data)

  #create a new dataframe with the text generated and concat to the original, then make a shuffle
  new = pd.DataFrame({'text':new_texts,'label':labels})
  df_new = shuffle(pd.concat([df_n,new]).reset_index(drop=True))
  print('Augmented shape of class',df_new.shape)
  return df_new

#create a list of dataframes
data_frames = []
#for each class, make the augmentation
for i in range(0,1):
  print('augmenting class ',i,"/23")
  data_frames.append(augment_text(train, i, 1))

train_augmented = pd.concat(data_frames)


#create a list of dataframes
data_frames = []
#for each class, make the augmentation
for i in range(0,24):
  print('augmenting class ',i,"/23")
  data_frames.append(augment_text(train, i, 200))

train_augmented = pd.concat(data_frames)
train_augmented.to_csv("agumented-test-data.csv", sep=',', encoding='utf-8', index=False)